In [1]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
beer_data = cPickle.load(open('../../beer_data/beer_vec_ds_df10.p', 'rb'))

Using TensorFlow backend.


[]


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

idxs = val_idxs
H = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = map(lambda s : list(idxs).index(s), a0)
    a1 = map(lambda s : list(idxs).index(s), a1)
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [4]:
train_X = beer_data.X[train_idxs]
train_X = map(lambda s : [beer_data.idx2word[t] for t in s if t > 0], train_X)

In [5]:
val_X = beer_data.X[val_idxs]
val_X = map(lambda s : [beer_data.idx2word[t] for t in s if t > 0], val_X)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False)
train_TF = tfidf.fit_transform(train_X)

In [6]:
val_TF = tfidf.transform(val_X)

In [13]:
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
scores = cosine_similarity(embedds_n)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [14]:
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.622527668018
0 0.630764199813
3 0.615854123934
2 0.619540610685


In [11]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=20)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(tokenizer=lambda i:i, lowercase=False)
train_C = count.fit_transform(train_X)
val_C = count.transform(val_X)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score

#for n_top in range(1, 30) :  
n_top = 4
lda = LatentDirichletAllocation(n_components=n_top)
train_lda = lda.fit_transform(train_C)
val_lda = lda.transform(val_C)

embedds_n = normalize(val_lda, 'l2')
nb_studies = len(val_idxs)
scores = np.dot(embedds_n, embedds_n.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000
scores = np.array(scores)

aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H['0'][i], scores[i])
print n_top, np.mean(aucs)

1 0.500170493183
2 0.710296810903
3 0.766128404167
4 0.828780794421
5 0.728363162594
6 0.733000582207
7 0.73845945943
8 0.725484990308
9 0.715338165464
10 0.741679627528
11 0.690483753801
12 0.739115287582
13 0.723561691851
14 0.737929461019
15 0.707615425239
16 0.74659927508
17 0.781842536399
18 0.709476549801
19 0.727780888787
20 0.716860596945
21 0.751282719178
22 0.747910954897
23 0.776697087493
24 0.652011112945
25 0.68041265337
26 0.721232803824
27 0.730539254943


In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score

#for n_top in range(1, 30) :  
n_top = 4
lda = LatentDirichletAllocation(n_components=n_top, n_jobs=6)
train_lda = lda.fit_transform(train_C)
val_lda = lda.transform(val_C)

embedds_n = normalize(val_lda, 'l2')
nb_studies = len(val_idxs)
scores = np.dot(embedds_n, embedds_n.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000
scores = np.array(scores)

for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.733605593562
0 0.73037470531
3 0.728938677461
2 0.73031096569


In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score

#for n_top in range(1, 30) :  
n_top = 23
lda = LatentDirichletAllocation(n_components=n_top, n_jobs=6)
train_lda = lda.fit_transform(train_C)
val_lda = lda.transform(val_C)

embedds_n = normalize(val_lda, 'l2')
nb_studies = len(val_idxs)
scores = np.dot(embedds_n, embedds_n.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000
scores = np.array(scores)

for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.699888570535
0 0.699681435342
3 0.696052804442
2 0.696945033317


In [34]:
top10 = ncount.argsort(axis=1)[:, -40:]

In [35]:
fnames=count.get_feature_names()

In [36]:
for l in top10 :
    for w in l :
        print fnames[w],
    print ""

taste more just so ... me at ! be had like 		have qqq on not one you n't as my 's for but was that in unk and beer is of to a this it , i . the 
bad " there qqq thin some much 's yellow smell n't 	in that very but i head this no light taste like : to - beer   unk with not it the of and is 		a , . 
as finish medium beer well aroma taste citrus unk carbonation light sweet good white on that but this hop malt qqq some to in nice   hops head : very - is of with 		the and , a . 
well good smooth beer taste it malt that on unk alcohol as roasted but black sweet nice this some coffee in qqq   to brown head very chocolate : - dark is with 		of the and a , . 


In [21]:
top10[:, -10:]

array([[ 6974,  8938, 13267,    82, 13143,  6986,    37,  6560,    55,
        13088],
       [ 8765,  6986, 13088,  8938,   560,  6974,     1,    82,    37,
           55],
       [   38,  6974,  8938, 14460,     1, 13088,   560,    37,    82,
           55],
       [ 3428,  6974, 14460,     1,  8938, 13088,   560,    82,    37,
           55]])